In [ ]:
import time
import sys
import os
import math
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import CellModeller
from scipy.interpolate import interp1d
plt.rcParams['figure.figsize'] = 7, 7

#### Funcs

In [ ]:
# continuous model
def step(p1, p2, p3, gamma, mu, d, e, n, Dt):
    # Update protein concs
    nextp1 = p1 + ((d + e*(p3**n))/(1 + p3**n) - gamma*p1 - mu*p1) * Dt
    nextp2 = p2 + ((d + e*(p1**n))/(1 + p1**n) - gamma*p2 - mu*p2) * Dt
    nextp3 = p3 + ((d + e*(p2**n))/(1 + p2**n) - gamma*p3 - mu*p3) * Dt
    return nextp1, nextp2, nextp3

In [ ]:
# Gillespie's

#### Build lineage's graph from last pickle

In [ ]:
# using Deg03 to check correct values of 
path = '../Data/'
model = 'simpleGrowth10/'
files = os.listdir(path+model)
files.sort()
# not using module (think on removing it)
#files = files[1:]

In [ ]:
# get last pickle's lineage
last_pickle = pickle.load(open(path+model+files[-1], 'rb'))

In [ ]:
# constructing data
lin = last_pickle['lineage']
nodes = list(lin.keys())
edges = [(v,k) for k,v in lin.items()]

In [ ]:
# create directed graph
G = nx.DiGraph()
# add cell's ids as nodes
G.add_node(1)
G.add_nodes_from(nodes)
# add (parent_id, child_id) tuples as edges
G.add_edges_from(edges)

In [ ]:
# just testing
list(G.successors(1))

### Loop over pickles

#### Sim params

In [ ]:
# Continuous model params
Dt = 0.05
gamma = 0.3
d = 198.
e = 0
n = 2
init_conds = [0, 0, 5.0]

# Sim params
pickeSteps = 1
#time between pickles 
p_time = Dt*pickeSteps #hours
#store_folder = path+model[:-1]+'SIM2/'

In [ ]:
# database file structure
"""
database = 
{
    'it': {
        'cell_id1':
        {'pos': [x, y, z], 'fluo': [p1, p2, p3]},
        'cell_id2':
        {'pos': [x, y, z], 'fluo': [p1, p2, p3]},
        ...
    },
}
"""

### loop

In [ ]:
# Initialize pickle 0 with desired initial conditions
pickle_0 = pickle.load(open(path+model+'step-00000.pickle', 'rb'))
pickle_0['cellStates'][1].color = init_conds
pos = pickle_0['cellStates'][1].pos
pos = [float(i) for i in pos]

database = {}
database[0] = {1: {'pos': pos, 'fluo': init_conds}}

#for idx in range(1, len(files)):
startproc = time.time()
for idx in range(1, len(files)):
    print("######################################################")
    print("idx: ", idx)
    startwhole = time.time()
    # get pickles lineage
    # this pickle needs to have concs, so get it from pickle save past iteration
    # Create new key in database:
    database[idx] = {}
    data1 = database[idx-1]
    data2 = pickle.load(open(path+model+files[idx], 'rb'))
    
    print(f"Number of cells: {len(data2['cellStates'].keys())}")
    
    # need to identify which cells divided from one pickle to the next
    # tells me which cells already divided when compared with previous pickle
    # will use: data['cellStates'].keys()

    # cells present in 2 but not in 1 (children)
    not_in_prev = list(set(data2['cellStates'].keys()) - set(data1.keys()))    
    
    # cell division
    if len(not_in_prev) > 0:
        # cells in 1 and not in 2 (parents)
        not_in_curr = list(set(data1.keys()) - set(data2['cellStates'].keys()))
        
        # each cell in not_in_curr (parents)
        for par_cell in not_in_curr:       
            succ = list(G.successors(par_cell))
            
            database[idx][succ[0]] = {}
            database[idx][succ[1]] = {}
            
            # SPLIT CONC from parent
            # 1) get conc from picke 1      
            concs_1 = data1[par_cell]['fluo']
            # 2) split conc in 2
            new_conc = np.array(concs_1)
            
            # 3) assign pos, conc to each child
            # Assign pos            
            # Needs to convert from numpy.float32 to float
            database[idx][succ[0]]['pos'] = [float(p) for p in data2['cellStates'][succ[0]].pos]
            database[idx][succ[1]]['pos'] = [float(p) for p in data2['cellStates'][succ[1]].pos]
            
            # Assign species conc
            database[idx][succ[0]]['fluo'] = new_conc.tolist()
            database[idx][succ[1]]['fluo'] = new_conc.tolist()
                    
            # cells remaining in not_in_prev obj
            new_left = list(set(not_in_prev) - set(succ))
                    
            # update not_in_prev
            not_in_prev = new_left
            
    # no cell division
    else:
        pass
    
    ##################################
    # RUN CONTINUOUS OR GILLESPIE SIM
    ##################################
    
    # this is for cells that did't get divided
    #step(p1, p2, p3, gamma, mu, d, e, n, Dt)
    
    cells_both = list(set(data2['cellStates'].keys()) & set(data1.keys()))
    start = time.time()
    # FOR EACH CELL
    for c in cells_both:
        database[idx][c] = {}
        # calculate concentrations
        pos_2 = [float(p) for p in data2['cellStates'][c].pos]
        concs = data1[c]['fluo']
        
        p1, p2, p3 = concs[0], concs[1], concs[2]
        mu = float(data2['cellStates'][c].effGrowth)                
        nextp1, nextp2, nextp3 = step(p1, p2, p3, gamma, mu, d, e, n, Dt)
        
        # assign to next pickle
        database[idx][c]['pos'] = pos_2
        database[idx][c]['fluo'] = [nextp1, nextp2, nextp3]                
    
    end = time.time()
    print(f"SIM: {end-start} secs")
    # STORE PICKLE WITH NEW CONCS
        ## Send picke2 (data2) to be read and stored
    # needs to have all concs updated
    
    endwhole = time.time()
    print(f"Time taken: {endwhole-startwhole}")
    
endproc = time.time()
print(f"Whole process took {endproc-startproc} secs")

In [ ]:
len(database)

#### save json

In [ ]:
start = time.time()
with open('data_contSIM100k.json', 'w') as outfile:  
    json.dump(database, outfile)
end = time.time()
print(f"saving took {end-start} secs")

#### save npy

In [ ]:
"""
start = time.time()
np.save("data_contSIM.npy", database)
end = time.time()
print(f"saving took: {end-start}")
"""

## PENDINGS

In [ ]:
# print number of cells: OK
# accumulate time and print OK
# timing of each section: sets, euler: OK
# try call step() once with vals for all of the cells: not too much time so no necessary. OK
# store pos, fluos in a numpy array instead of pickles: OK
# CHANGE COLOR FOR CONCENTRATION
# check problem when store json with dict keys as integers and loaded as strings

# SIMS
# for gamma = 1, change timestep from 0.05 (?): All kept dt = 0.05
# figure out how many cells for each simulation: OK
# do sims for gamma = 1, 5, 10
# change names to pickles, instead of timestep, put number of cells

#### Load file

In [ ]:
## When I store a file with dict keys as integers, they come as strings when loaded

In [ ]:
with open('data_contSIM50k.json') as json_file:  
    database = json.load(json_file)

In [ ]:
len(database)

In [ ]:
database['0']

In [ ]:
start = time.time()
database_kymo = []
width = 5
for i in np.arange(10, len(database), 10):
    # when used from database file generated in the code, so keys are integers
    #data = database[i]
    # when file is loaded, key are strings
    data = database[str(i)]
    
    pos_x = [data[k]['pos'][0] for k in data.keys()]
    pos_y = [data[k]['pos'][1] for k in data.keys()]
    
    xmax = math.ceil(max(list(map(abs, pos_x))))
    ymax = math.ceil(max(list(map(abs, pos_y))))
    
    grid_size = max(xmax, ymax)
    xx = np.arange(grid_size)
    yy = np.arange(grid_size)
    y,x = np.meshgrid(xx,yy)
    c = grid_size / 2 - 1/2, grid_size / 2-1/2
    r = np.sqrt((x-c[0])**2 + (y-c[1])**2)
    
    r_cells = np.array([np.sqrt(data[k]['pos'][0]**2 + data[k]['pos'][1]**2) for k in data.keys()])
    R_cells = np.array([data[k]['fluo'][0] for k in data.keys()])
    G_cells = np.array([data[k]['fluo'][1] for k in data.keys()])
    B_cells = np.array([data[k]['fluo'][2] for k in data.keys()])
    
    nbins = int(r.max() // width)
    bins_acc = []
    R_acc = []
    G_acc = []
    B_acc = []
    for dr in range(nbins):
        bins_acc.append(dr*width)

        idx = np.where((r_cells > dr*width)*(r_cells < (dr+1)*width))
        R_acc.append(np.mean(R_cells[idx]))
        G_acc.append(np.mean(G_cells[idx]))
        B_acc.append(np.mean(B_cells[idx]))

    database_kymo.append([bins_acc, R_acc, G_acc, B_acc])

end = time.time()
print(f"creating kymo database took {end-start} secs")

In [ ]:
x = [d[0] for d in database_kymo]
R = [d[1] for d in database_kymo]
G = [d[2] for d in database_kymo]
B = [d[3] for d in database_kymo]

In [ ]:
start = time.time()
kymo = np.zeros([len(database_kymo), len(x[-1]), 3])
for i, x_vals in enumerate(x):
    kymo[i, :len(x_vals), 0] = R[i]
    kymo[i, :len(x_vals), 1] = G[i]
    kymo[i, :len(x_vals), 2] = B[i]

vals = kymo[:,:,0]
kymo[:,:,0] = (vals-vals.min()) / (vals.max()-vals.min())

vals = kymo[:,:,1]
kymo[:,:,1] = (vals-vals.min()) / (vals.max()-vals.min())

vals = kymo[:,:,2]
kymo[:,:,2] = (vals-vals.min()) / (vals.max()-vals.min())

end = time.time()
print(f"creating kymo database took {end-start} secs")

In [ ]:
plt.imshow(kymo);

In [ ]:
with open('data_contSIM100k.json', 'w') as outfile:  
    json.dump(kymo, outfile)

In [ ]:
np.save("kymo50k.npy", kymo)